Analýza dát
===========
Zámerom nášho projektu je natrénovať konvolučnú neurónovú sieť, aby dokázala filtrovať zašumený obraz. Náš dataset predstavuje množina dvojíc zašumený obraz-bezchybný obraz.

## Pôvod datasetu
Použitý dataset pozostáva zo skupiny 200 dvojíc obrázkov zachytávajúcich rôzne scény, vyhotovených na pätici rôznych smartfónov. Celý dataset je dostupný na tejto [stránke](https://www.eecs.yorku.ca/~kamel/sidd/dataset.php).

## Ukážky
TODO
aa
## Voláke grafíky?

## Validácia
??

## Spôsov vyhodnocovania
V rámci objektívneho vyhodnocovania výsledkov našej neurónovej siete pri rôznych nastaveniach hyperparametrov využijeme absolútny rozdiel pixelových hodnôt bezchybného obrazu na vstupe a výsledného obrazu po predikcii našou neurónovou sieťou. Vizualizácia tohto rozdielu nám pomôže odhaliť, ktoré prvky či javy obrazu robia sieti problémy a s ktorými sa naopak dokáže vysporiadať.

In [1]:
import os
import re
from datetime import datetime

import math
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras

from models import DnCNN, dcnn_loss

os.environ['CUDA_VISIBLE_DEVICES'] = "0"
PATCH_SIZE = 140
SHUFFLE_BUFFER_SIZE = 100
TEST_SIZE = 200

In [2]:
def load_image_data(scenes=None):
    def decode_img(img):
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.convert_image_dtype(img, tf.float32, True)
        img = tf.expand_dims(img, 0)
        img = tf.image.extract_patches(img,
                                       [1, PATCH_SIZE, PATCH_SIZE, 1],
                                       [1, PATCH_SIZE, PATCH_SIZE, 1],
                                       [1, 1, 1, 1],
                                       padding='VALID')
        img = tf.reshape(img, [-1, PATCH_SIZE, PATCH_SIZE, 3])
        return img

    def process_path(file_path):
        noisy = tf.io.read_file(path + file_path + '/NOISY_SRGB_010.PNG')
        gt = tf.io.read_file(path + file_path + '/GT_SRGB_010.PNG')
        noisy = decode_img(noisy)
        gt = decode_img(gt)
        return noisy, gt

    path = 'datasets/SIDD_Small_sRGB_Only/SIDD_Small_sRGB_Only/Data/'
    f = open('datasets/SIDD_Small_sRGB_Only/SIDD_Small_sRGB_Only/Scene_Instances.txt')
    scene_instances = f.readlines()
    paths = None
    if scenes is not None:
        regex = ''
        for scene in scenes:
            regex = regex + '_%03d_|' % scene
        regex = regex[:-1]
        scene_instances = [i for i in scene_instances if re.search(regex, i)]
        paths = tf.data.Dataset.from_tensors(scene_instances)

    paths = tf.data.TextLineDataset('datasets/SIDD_Small_sRGB_Only/SIDD_Small_sRGB_Only/Scene_Instances.txt')

    x = paths.map(process_path)
    return x


def image_to_patches(image):
    image = tf.expand_dims(image, 0)
    image_patches = tf.image.extract_patches(image,
                                       [1, PATCH_SIZE, PATCH_SIZE, 1],
                                       [1, PATCH_SIZE, PATCH_SIZE, 1],
                                       [1, 1, 1, 1],
                                       padding='SAME')
    image_patches = tf.reshape(image_patches, [-1, PATCH_SIZE, PATCH_SIZE, 3])
    return image_patches


def use_predict_on_patches(image_patches, model):
    predicted_patches = []

    for i in range(len(image_patches)):
        predicted_patches.append(model.predict(np.expand_dims(image_patches[i], 0)))
        print('Prediction done ' + str(image_patches[i].shape))
    return predicted_patches


def patches_to_image(patches, height, width):
    num_of_patches_vertical = math.ceil(height / PATCH_SIZE)
    num_of_patches_horizontal = math.ceil(width / PATCH_SIZE)

    print('Height is: ' + str(height))
    print('Width is: ' + str(width))

    print('Vertical number of patches is: ' + str(num_of_patches_vertical))
    print('Horizontal number of patches is: ' + str(num_of_patches_horizontal))

    pad = [[0, 0], [0, 0]]

    reconstructed_patches = tf.reshape(patches, [1, num_of_patches_vertical, num_of_patches_horizontal, PATCH_SIZE * PATCH_SIZE, 3])
    reconstructed_patches = tf.split(reconstructed_patches, PATCH_SIZE * PATCH_SIZE, 3)
    reconstructed_patches = tf.stack(reconstructed_patches, axis=0)
    reconstructed_patches = tf.reshape(reconstructed_patches, [PATCH_SIZE * PATCH_SIZE, num_of_patches_vertical, num_of_patches_horizontal, 3])

    result = tf.batch_to_space(reconstructed_patches, [PATCH_SIZE, PATCH_SIZE], pad)
    return result


In [3]:
x = load_image_data()

In [4]:
x = x.unbatch()
x = x.batch(batch_size=10)
shuffled_data = x.shuffle(buffer_size=SHUFFLE_BUFFER_SIZE)
test = shuffled_data.take(TEST_SIZE).repeat()
train = shuffled_data.skip(TEST_SIZE).repeat()

In [ ]:
model = DnCNN(depth=20)
model.compile(optimizer=keras.optimizers.Adam(), loss=dcnn_loss, metrics=['accuracy'])

now = datetime.now()
tensorboard_callback = keras.callbacks.TensorBoard(
    log_dir='logs\log_from_{}'.format(now.strftime("%Y-%m-%d_at_%H-%M-%S")),
    histogram_freq=1)

model.fit(x=train, steps_per_epoch=3, validation_data=test, epochs=10, validation_steps=10, callbacks=[tensorboard_callback])
model.summary()

Train for 3 steps, validate for 10 steps
Epoch 1/10
3/3 [==============================] - 11s 4s/step - loss: 82555.3242 - accuracy: 0.4014 - val_loss: 207.3288 - val_accuracy: 0.8483
Epoch 2/10
3/3 [==============================] - 4s 1s/step - loss: 9061.1193 - accuracy: 0.5263 - val_loss: 239.0855 - val_accuracy: 0.9061
Epoch 3/10
2/3 [===================>..........] - ETA: 0s - loss: 1937.3776 - accuracy: 0.2524

In [ ]:
now = datetime.now()
model.save('models\model_made_on_{}'.format(now.strftime("%Y-%m-%d_at_%H-%M-%S")))

In [ ]:
loaded = keras.models.load_model('models/model_made_on_2019-11-16_at_02-03-01', compile=False)
loaded.compile(optimizer=keras.optimizers.Adam(), loss=dcnn_loss, metrics=['accuracy'])
image = mpimg.imread('datasets/SIDD_Small_sRGB_Only/SIDD_Small_sRGB_Only/Data/0118_006_N6_00100_00025_3200_L/NOISY_SRGB_010.PNG')

image_patches = image_to_patches(image)

reconstructed_patches = use_predict_on_patches(image_patches, loaded)

In [ ]:
joined_image = patches_to_image(image_patches, image.shape[0], image.shape[1])
reconstructed_image = patches_to_image(reconstructed_patches, image.shape[0], image.shape[1])

plt.figure()
plt.imshow(joined_image[0])
plt.show()

plt.figure()
plt.imshow(reconstructed_image[0])
plt.show()